In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [93]:
vac = pd.read_csv("data/2000-2015 CAKindergartenData.csv")


In [96]:
# Create new dataframe with info needed to calculat Perusiss vaccination rate for each county by year

dtp_df = vac[['Year', 'COUNTY', 'ENROLLMENT', 'DTP#']]
dtp_df = dtp_df[dtp_df['Year'] != 2000]
dtp_df.head()

,Year,COUNTY,ENROLLMENT,DTP#
7418,2001,ALAMEDA,12,12
7419,2001,ALAMEDA,78,77
7420,2001,ALAMEDA,77,76
7421,2001,ALAMEDA,56,54
7422,2001,ALAMEDA,41,41


In [97]:
dtp_df[dtp_df["ENROLLMENT"] == " "]["ENROLLMENT"].count()

1366

In [98]:
dtp_df.shape

(110383, 4)

In [99]:
dtp_df = dtp_df[dtp_df["ENROLLMENT"] != " "]
dtp_df[dtp_df["ENROLLMENT"] == " "]["ENROLLMENT"].count()

dtp_df["ENROLLMENT"] = pd.to_numeric(dtp_df["ENROLLMENT"])
dtp_df.shape

(109017, 4)

In [100]:
dtp_df[dtp_df['DTP#'] == " "].count()
dtp_df['DTP#'] = pd.to_numeric(dtp_df['DTP#'])
# dtp_df.dtypes

In [101]:
# sum up enrollement and vaccination numbers by year and county

dtp_summed_df = dtp_df.groupby(['Year','COUNTY']).sum()
dtp_summed_df['cnty_vac_rate'] = 0
dtp_summed_df.head(10)

dtp_summed_df.shape

(868, 3)

In [102]:
dtp_summed_df['cnty_vac_rate'] = dtp_summed_df['DTP#']/dtp_summed_df['ENROLLMENT'] * 100

dtp_summed_df.head(10)

ENROLLMENT     DTP#  cnty_vac_rate
Year COUNTY                                          
2001 ALAMEDA          19392.0  18332.0      94.533828
     AMADOR             345.0    336.0      97.391304
     BUTTE             2508.0   2358.0      94.019139
     CALAVERAS          399.0    373.0      93.483709
     COLUSA             322.0    315.0      97.826087
     CONTRA COSTA     13544.0  13055.0      96.389545
     DEL NORTE          298.0    292.0      97.986577
     EL DORADO         2059.0   1942.0      94.317630
     FRESNO           14320.0  13932.0      97.290503
     GLENN              418.0    411.0      98.325359

In [103]:
pop_df = pd.read_csv("data/CA County Population Over Time All.csv", header=None)
pop_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,table with row headers in column A and column ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geographic Area,NaN,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
2,California,"33,871,653","33,987,977","34,479,458","34,871,843","35,253,159","35,574,576","35,827,943","36,021,202","36,250,311",...,"36,961,229","37,254,518","37,327,690","37,672,654","38,019,006","38,347,383","38,701,278","39,032,444","39,296,476","39,459,622"
3,.Alameda County,"1,443,705","1,449,840","1,469,484","1,462,106","1,454,163","1,445,721","1,441,545","1,444,484","1,455,715",...,"1,498,539","1,510,261","1,513,402","1,532,215","1,556,648","1,582,936","1,611,572","1,637,141","1,653,236","1,663,190"
4,.Alpine County,"1,206","1,209","1,196","1,237","1,233","1,263","1,219","1,276","1,274",...,"1,197","1,175","1,161","1,093","1,110","1,127","1,084","1,088","1,057","1,120"


In [104]:
pop_df.index
pop_df.shape

(61, 21)

In [105]:
pop_df = pop_df[1:]

pop_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
1,Geographic Area,NaN,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
2,California,"33,871,653","33,987,977","34,479,458","34,871,843","35,253,159","35,574,576","35,827,943","36,021,202","36,250,311",...,"36,961,229","37,254,518","37,327,690","37,672,654","38,019,006","38,347,383","38,701,278","39,032,444","39,296,476","39,459,622"
3,.Alameda County,"1,443,705","1,449,840","1,469,484","1,462,106","1,454,163","1,445,721","1,441,545","1,444,484","1,455,715",...,"1,498,539","1,510,261","1,513,402","1,532,215","1,556,648","1,582,936","1,611,572","1,637,141","1,653,236","1,663,190"
4,.Alpine County,"1,206","1,209","1,196","1,237","1,233","1,263","1,219","1,276","1,274",...,"1,197","1,175","1,161","1,093","1,110","1,127","1,084","1,088","1,057","1,120"
5,.Amador County,"35,101","35,153","35,697","36,561","36,994","37,400","38,040","38,353","38,756",...,"38,238","38,091","37,884","37,539","37,112","36,635","36,748","37,047","37,473","38,626"


In [106]:
pop_df.loc[0:1, 0] = "COUNTY"
pop_df.loc[0:1, 1] = '1999'
pop_df.iloc[0]
pop_df.columns = pop_df.iloc[0]
pop_df = pop_df[2:]
pop_df.head()


1,COUNTY,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
3,.Alameda County,"1,443,705","1,449,840","1,469,484","1,462,106","1,454,163","1,445,721","1,441,545","1,444,484","1,455,715",...,"1,498,539","1,510,261","1,513,402","1,532,215","1,556,648","1,582,936","1,611,572","1,637,141","1,653,236","1,663,190"
4,.Alpine County,"1,206","1,209","1,196","1,237","1,233","1,263","1,219","1,276","1,274",...,"1,197","1,175","1,161","1,093","1,110","1,127","1,084","1,088","1,057","1,120"
5,.Amador County,"35,101","35,153","35,697","36,561","36,994","37,400","38,040","38,353","38,756",...,"38,238","38,091","37,884","37,539","37,112","36,635","36,748","37,047","37,473","38,626"
6,.Butte County,"203,125","203,807","206,193","209,120","211,481","213,065","214,752","216,824","217,469",...,"219,777","220,002","219,957","220,003","220,969","221,768","223,629","224,818","226,525","229,294"
7,.Calaveras County,"40,535","40,645","41,304","42,380","43,755","44,530","45,295","46,045","46,268",...,"45,900","45,578","45,470","45,163","44,826","44,667","44,677","44,954","45,315","45,670"


In [107]:
pop_df.drop(["1999", "2000", "2016", "2017", "2018"], axis=1, inplace=True )

In [108]:
pop_df["COUNTY"] = pop_df["COUNTY"].str.split(' County', expand=True)
pop_df["COUNTY"] = pop_df["COUNTY"].str.split('.', expand=True)[1]
pop_df["COUNTY"] = pop_df["COUNTY"].str.split(' County', expand=True)[0]


In [109]:
pop_df['COUNTY'] = pop_df['COUNTY'].str.upper()

pop_df.head()

1,COUNTY,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
3,ALAMEDA,"1,469,484","1,462,106","1,454,163","1,445,721","1,441,545","1,444,484","1,455,715","1,477,208","1,498,539","1,510,261","1,513,402","1,532,215","1,556,648","1,582,936","1,611,572"
4,ALPINE,"1,196","1,237","1,233","1,263","1,219","1,276","1,274","1,202","1,197","1,175","1,161","1,093","1,110","1,127","1,084"
5,AMADOR,"35,697","36,561","36,994","37,400","38,040","38,353","38,756","38,248","38,238","38,091","37,884","37,539","37,112","36,635","36,748"
6,BUTTE,"206,193","209,120","211,481","213,065","214,752","216,824","217,469","219,034","219,777","220,002","219,957","220,003","220,969","221,768","223,629"
7,CALAVERAS,"41,304","42,380","43,755","44,530","45,295","46,045","46,268","46,265","45,900","45,578","45,470","45,163","44,826","44,667","44,677"


In [110]:
long_df = pd.melt(pop_df, id_vars=['COUNTY'], value_name = 'population')

In [111]:
long_df[1] = long_df[1].astype('int64')
print(long_df.dtypes)
long_df[1]
long_df.head()
# int(a.replace(',' , ''))

COUNTY        object
1              int64
population    object
dtype: object


,COUNTY,1,population
0,ALAMEDA,2001,"1,469,484"
1,ALPINE,2001,"1,196"
2,AMADOR,2001,"35,697"
3,BUTTE,2001,"206,193"
4,CALAVERAS,2001,"41,304"


In [112]:
long_df.set_index([1, 'COUNTY'], inplace=True)

In [113]:
new_df = pd.merge(dtp_summed_df, long_df,  how='left', left_on=['Year','COUNTY'], right_on = [1,'COUNTY'])

new_df.head()

ENROLLMENT     DTP#  cnty_vac_rate population
Year COUNTY                                                  
2001 ALAMEDA       19392.0  18332.0      94.533828  1,469,484
     AMADOR          345.0    336.0      97.391304     35,697
     BUTTE          2508.0   2358.0      94.019139    206,193
     CALAVERAS       399.0    373.0      93.483709     41,304
     COLUSA          322.0    315.0      97.826087     19,022

In [114]:
new_df['population'] = new_df['population'].str.replace(',', '')
new_df['population'] = new_df['population'].astype(float)

new_df.head()

ENROLLMENT     DTP#  cnty_vac_rate  population
Year COUNTY                                                   
2001 ALAMEDA       19392.0  18332.0      94.533828   1469484.0
     AMADOR          345.0    336.0      97.391304     35697.0
     BUTTE          2508.0   2358.0      94.019139    206193.0
     CALAVERAS       399.0    373.0      93.483709     41304.0
     COLUSA          322.0    315.0      97.826087     19022.0

In [115]:
url="https://data.chhs.ca.gov/api/3/action/datastore_search?resource_id=a7a4d868-f6d2-49c9-8a11-904c69a0e3a0&limit=8000"

response = requests.get(url)
response_json = response.json()

records = response_json['result']['records']

In [116]:
outbreak_df = pd.DataFrame(records)
outbreak_df = outbreak_df[outbreak_df['disease'] == 'Diphtheria'][['count', 'county', 'year']]
outbreak_df = outbreak_df[(outbreak_df['year'] != '2016') & (outbreak_df['year'] != '2017')]
outbreak_df.head()



,count,county,year
0,0,Alameda,2001
1,0,Alameda,2002
2,0,Alameda,2003
3,0,Alameda,2004
4,0,Alameda,2005


In [117]:
outbreak_df['year'] = outbreak_df['year'].astype('int64')
outbreak_df['county'] = outbreak_df['county'].str.upper()

outbreak_df.set_index(['year', 'county'], inplace=True)

In [121]:
outbreak_df.index.unique(level=0)

Int64Index([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
            2012, 2013, 2014, 2015],
           dtype='int64', name='year')

In [139]:
new_df.index.unique(level=0)
final_df = new_df.merge(outbreak_df, left_on=['Year', 'COUNTY'], right_index=True)
final_df['outbreak%'] = final_df['count']/final_df['population']
final_df.head()

ENROLLMENT     DTP#  cnty_vac_rate  population  count  \
Year COUNTY                                                             
2001 ALAMEDA       19392.0  18332.0      94.533828   1469484.0      0   
     AMADOR          345.0    336.0      97.391304     35697.0      0   
     BUTTE          2508.0   2358.0      94.019139    206193.0      0   
     CALAVERAS       399.0    373.0      93.483709     41304.0      0   
     COLUSA          322.0    315.0      97.826087     19022.0      0   

                outbreak%  
Year COUNTY                
2001 ALAMEDA          0.0  
     AMADOR           0.0  
     BUTTE            0.0  
     CALAVERAS        0.0  
     COLUSA           0.0